# Successful-Hollywood-Movies
## 1. Introduction
This project is based on a Competition I participated in through DataCamp. The problem statement was as follows: You have just been hired by a large movie studio to perform data analysis. Your manager, an executive at the company, wants to make new movies that "recapture the magic of old Hollywood." So you've decided to look at the most successful films that came out before Titanic in 1997 to identify patterns and help generate ideas that could turn into future successful films.

In this project I utilized SQL to extract ... 
and address the following questions:
1. How many movies are present in the database?
2. There seems to be a lot of missing data in the gross and budget columns. How many rows have missing data? What would you recommend your manager to do with these rows? 
3. How many different certifications or ratings are present in the database? 
4. What are the top five countries in terms of number of movies produced?
5. What is the average duration of English versus French movies?
6. Any other insights you found during your analysis? 

## 2. Data
The dataset containing Movies records was collected from [IMBD](https://www.imdb.com/).
The following table gives a brief description of the columns and key variables.
| Column name | Description |
|---|---|
| id | Unique movie identifier. |
| title | The title of the movie. |
| release_year | The year the movie was released to the public. |
| country | The country in which the movie was released. |
| duration | The runtime of the movie, in minutes. |
| language | The original language the movie was produced in. |
| certification | The [rating](https://simple.wikipedia.org/wiki/Motion_Picture_Association_film_rating_system) the movie was given based on their suitability for audiences. |
| gross | The revenue the movie generated at the box office, in USD. |
| budget | The available budget the production had for producing the movie, in USD. |

### 2.1 Initial Data Exploration
Create a connection to SQL, and using SQL Magic to run SQL commands 

In [33]:
!pip install ipython-sql


In [64]:
import pandas as pd

#Load the CSV file into a DataFrame
df = pd.read_csv('films.csv')
reviews = pd.read_csv('reviews.csv')

In [65]:
# Load pandas DataFrame into an in-memory SQLite database
import sqlite3

# Create an in-memory SQLite connection
conn = sqlite3.connect('cinema.db')

# Write the DataFrame to an SQLite table
df.to_sql('films', conn, index=False, if_exists='replace')
reviews.to_sql('reviews', conn, index=False, if_exists='replace')

# Load the sql extension
%reload_ext sql

# Use SQL magic to query the table
%sql sqlite:///cinema.db


In [39]:
%%sql
SELECT * FROM films LIMIT 10;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


id,title,release_year,country,duration,language,certification,gross,budget
1,Intolerance: Love's Struggle Throughout the Ages,1916.0,USA,123.0,None,Not Rated,None,385907.0
2,Over the Hill to the Poorhouse,1920.0,USA,110.0,None,None,3000000.0,100000.0
3,The Big Parade,1925.0,USA,151.0,None,Not Rated,None,245000.0
4,Metropolis,1927.0,Germany,145.0,German,Not Rated,26435.0,6000000.0
5,Pandora's Box,1929.0,Germany,110.0,German,Not Rated,9950.0,None
6,The Broadway Melody,1929.0,USA,100.0,English,Passed,2808000.0,379000.0
7,Hell's Angels,1930.0,USA,96.0,English,Passed,None,3950000.0
8,A Farewell to Arms,1932.0,USA,79.0,English,Unrated,None,800000.0
9,42nd Street,1933.0,USA,89.0,English,Unrated,2300000.0,439000.0
10,She Done Him Wrong,1933.0,USA,66.0,English,Approved,None,200000.0


#### Identifying outliers

### 2.2 Missing Data
Number of missing values for each column

In [ ]:
%%sql
SELECT 
  SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS id_missing,
  SUM(CASE WHEN title IS NULL THEN 1 ELSE 0 END) AS title_missing,
  SUM(CASE WHEN country IS NULL THEN 1 ELSE 0 END) AS country_missing,
  SUM(CASE WHEN duration IS NULL THEN 1 ELSE 0 END) AS duration_missing,
  SUM(CASE WHEN language IS NULL THEN 1 ELSE 0 END) AS language_missing,
  SUM(CASE WHEN certification IS NULL THEN 1 ELSE 0 END) AS certification_missing,
  SUM(CASE WHEN gross IS NULL THEN 1 ELSE 0 END) AS gross_missing,
  SUM(CASE WHEN budget IS NULL THEN 1 ELSE 0 END) AS budget_missing

FROM films;

## 3. Methodology

## 4. Results
### 4.1. Number of Movies Included in the Dataset

**Excluding the duplicated records, there are 4,846 movies presented in this database** <br>
The code below verifies that there are two movies with the exact same title but in reality they are different movies, so it is important to verify exact duplicates by including "duration","release_year" and "country" columns:

In [63]:
%%sql
SELECT COUNT(*) AS total_rows FROM films;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


total_rows
4968


In [46]:
%%sql
SELECT     SUM(duplicate_count) AS total_duplicate_count
FROM (
SELECT title, release_year, duration, country, COUNT(*)-1 AS duplicate_count
FROM films
GROUP BY title, release_year, duration, country
HAVING COUNT(*) > 1
) AS duplicated_counts

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


total_duplicate_count
122


In [58]:
%%sql

WITH duplicate_titles AS (
    SELECT title, duration, gross, certification, budget, language, country
    FROM films
    GROUP BY title
    HAVING COUNT(*) > 1
)

-- Find rows with those duplicate titles and ensure other columns are not identical
SELECT f1.*
FROM films f1
JOIN duplicate_titles dt ON f1.title = dt.title
WHERE EXISTS (
    SELECT 1
    FROM films f2
    WHERE f1.title = f2.title
      AND (f1.duration != f2.duration
        OR f1.gross != f2.gross
        OR f1.certification != f2.certification
        OR f1.budget != f2.budget
        OR f1.language != f2.language
        OR f1.country != f2.country)
)
ORDER BY f1.title;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


id,title,release_year,country,duration,language,certification,gross,budget
271,Out of the Blue,1980.0,Canada,94.0,English,R,None,2200000.0
2644,Out of the Blue,2006.0,New Zealand,100.0,English,None,728.0,None
2704,The Host,2006.0,South Korea,110.0,Korean,R,2201412.0,12215500000.0
4295,The Host,2013.0,USA,125.0,English,PG-13,26616999.0,40000000.0


### 4.2. Gross and Budget Columns Missing Data Handling
**From the main data table there are 1,076 rows that miss data either in gross or budget column. That's about 22% of the data which is significant. Among the 917 records for movies that are released before 1997, the number of rows missing gross revenue data is 242 which is a large portion of the data as well. For these records where the gross revenue is missing, we could join the 'film' table with 'review' table and create a column which calculates the success of each movie based on number of votes and imdb score of that movie. This would help find the successful movies before 1997 for which the gross revenue data is not available.** <br>
<br>
**Among the movies that are released prior to 1997 and the gross revenue data is not available for them, top 5 which has recieved a combination of highest number of votes and best imdb scores are: <br>
1- The shining <br>
2- Taxi Driver <br>
3- 12 Angry Men <br>
4- Casablanca <br>
5- Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb**<br>
<br>
Number of rows that either 'gross' or 'budget' columns are missing

In [62]:
%%sql
SELECT 
  SUM(CASE WHEN (gross IS NULL OR budget IS NULL) THEN 1 ELSE 0 END) AS gross_or_budget_missing,
  SUM(CASE WHEN (release_year < 1997 AND gross IS NULL) THEN 1 ELSE 0 END) AS gross_missing_before_1997 --Number of rows that 'gross' column is missing for movies that are released before 1997
  FROM films;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


gross_or_budget_missing,gross_missing_before_1997
1076,242


For the rows that the gross value is missing, we can create a new metric, "movie_rating" which is calculated by multiplying number of reviews by imbd score of that movie. This is to avoid bias if there are only few reviews for a specific movie.

In [71]:
%%sql
SELECT *, reviews.imdb_score*reviews.num_votes AS movie_rating
FROM films
LEFT JOIN reviews ON films.id = reviews.film_id
	WHERE films.release_year <1997
	AND gross IS NULL
	ORDER BY movie_rating DESC
LIMIT 5
;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


id,title,release_year,country,duration,language,certification,gross,budget,id_1,film_id,num_user,num_critic,imdb_score,num_votes,facebook_likes,movie_rating
284,The Shining,1980.0,USA,146.0,English,R,None,19000000.0,1055,284,1320.0,253.0,8.4,610333,37000,5126797.2
209,Taxi Driver,1976.0,USA,110.0,English,R,None,1300000.0,813,209,881.0,211.0,8.3,507063,35000,4208622.9
69,12 Angry Men,1957.0,USA,96.0,English,Not Rated,None,350000.0,3893,69,888.0,177.0,8.9,447785,40000,3985286.5
29,Casablanca,1942.0,USA,82.0,English,PG,None,950000.0,4968,29,1123.0,242.0,8.6,387508,23000,3332568.8
101,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,1964.0,USA,95.0,English,PG,None,1800000.0,2921,101,839.0,192.0,8.5,342585,18000,2911972.5


### 4.3. Number of Movie Certifications
**There are 12 different certifications present in this database and 302 rows where the info for certification is missing.** <br>
Based on the historical MPAA (Motion Picture Association of America) ratings, these ratings have changed and modified over time but the following ratings approxoimately refer to the same category of movies:

Approved/Passed (used prior to 1968 for G certified movies)
M(used for 1968-1970) and GP (used for 1970-1972) for PG certification
X (used prior to 1990 for NC-17 movies)

Unrated/Not Rated/Movies without certification data will not be used to draw insight from certification. <br>
We will replace the afformentioned certifications with it's modern MPAA equivalen.

In [73]:
%%sql
SELECT certification, COUNT(*)
FROM films
GROUP BY certification
Order By COUNT(*) DESC;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


certification,COUNT(*)
R,2118
PG-13,1462
PG,701
None,302
Not Rated,116
G,112
Unrated,62
Approved,55
X,13
Passed,9


In [76]:
%%sql
WITH updated_certifications AS (
    SELECT
        title,
        release_year,
        duration,
        gross,
        budget,
        language,
        country,
        CASE
            WHEN certification IN ('Approved', 'Passed') THEN 'G'
            WHEN certification IN ('M', 'GP') THEN 'PG'
            WHEN certification = 'X' THEN 'NC-17'
            ELSE certification
        END AS certification
    FROM films
)
-- Now we can query the CTE as if it were a regular table
SELECT certification, 
	   ROUND(AVG(gross)) AS avg_gross, 
	   ROUND(MIN(gross)) AS min_gross, 
	   ROUND(MAX(gross)) AS max_gross, 
	   COUNT(*),
	   ROUND(AVG((CAST(gross AS numeric)/CAST (budget AS numeric))*100))AS BER
FROM updated_certifications
WHERE gross IS NOT NULL
-- Remove outliers like Tarnation with very limited budget
	AND budget >=5000
-- WHERE certification NOT IN ('','Not Rated','Unrated')
GROUP BY certification
Order By COUNT(*) DESC
;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


certification,avg_gross,min_gross,max_gross,COUNT(*),BER
R,31587246.0,162.0,363024263.0,1736,921.0
PG-13,68880734.0,1111.0,936627416.0,1332,182.0
PG,76248547.0,4756.0,474544677.0,578,374.0
G,76001633.0,14873.0,422783777.0,111,550.0
None,851197.0,2245.0,7518876.0,51,113.0
Not Rated,2578859.0,1332.0,36000000.0,42,317.0
Unrated,5790881.0,18435.0,43650000.0,23,207.0
NC-17,13340584.0,126387.0,44793200.0,16,539.0


The transformation of data above simplifies the data analytics process. We see majority of movies are classified as Restricted content. Although the average gross revenue of the family friendly movies (G, PG and PG-13 certification) is the highest among all movie categories which reflect it's market dominance and scale, this doesn't necessarily mean that it is the best investment. Budget Efficiency Ratio (BER) of Restricted movies is ~67% higher than G certified movies, which reflects it's potential for profitability and explains why producers have created most movies in this category. 

### 4.4. Countries with Highest Number of Movies Produced
**This list ranks the top 5 countries by the number of movies produced:<br>
1- USA <br>
2- UK <br>
3- France <br>
4- Canada <br>
5- Germany**

In [86]:
%%sql
SELECT DISTINCT country, COUNT(*) AS movie_count
FROM films
GROUP BY country
ORDER BY movie_count DESC
LIMIT 5;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


country,movie_count
USA,3750
UK,443
France,153
Canada,123
Germany,97


### 4.5. Average Duration of English vs. French Movies
**Average English movie duration is 107.6 min versus 104.7 min for French movies.**

In [90]:
%%sql
SELECT ROUND(AVG(duration),2) AS avg_duration, language
FROM films
WHERE language IN ('English', 'French')
GROUP BY language;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


avg_duration,language
107.64,English
104.67,French


### 4.6. Other Insights
**There are movies released originally in 47 different countries in this table. On average the movies released in New Zealand, Taiwan and Peru (Despite the fact of having overal very limited number of movies) have the highest gross revenues.**

In [94]:
%%sql
SELECT DISTINCT country, ROUND(AVG(gross),2) AS avg_gross, COUNT(gross) 
FROM films
WHERE gross IS NOT NULL
GROUP BY country
ORDER BY avg_gross DESC
LIMIT 10;

   sqlite:///:memory:
 * sqlite:///cinema.db
   sqlite:///my_database.db
Done.


country,avg_gross,COUNT(gross)
New Zealand,108180043.5,12
Taiwan,64340682.0,2
Peru,57362581.0,1
USA,55476089.19,3237
South Africa,53580080.33,3
Australia,40205909.57,42
Japan,33553701.47,17
UK,32440108.68,364
Germany,27984127.66,88
Canada,22432066.68,72
